In [2]:
import gc
from tqdm.notebook import tqdm
import scipy
import scipy.sparse

import os
import pandas as pd
import scipy.sparse
import pandas as pd
import numpy as np

from sklearn.model_selection import KFold
import pickle
import bz2

Competition's data was strongly anonymizer. Continuous features was transformed into categorical through histogram splitting. As a result each feature has no more than 30 values. We use this property in order to convert data into text files where each sample is a string. That format will allow us reshuffle data between neural net's epoches without high RAM load as unarchived data allocates a lot of it.

In [4]:
if os.path.isdir("nn_data"):
    shutil.rmtree("nn_data")

os.mkdir("nn_data")
os.mkdir(os.path.join("nn_data", "clear_test"))
os.mkdir(os.path.join("nn_data", "clear"))
os.mkdir(os.path.join("nn_data", "test"))

In [5]:
def credit2string(df, id2latters, feature_columns ):
    return ' '.join((''.join((id2latters[a] for a in df[ft].tolist())) for ft in feature_columns))

In [6]:
import string
latters = list(string.ascii_uppercase)+list(string.ascii_lowercase)
latters2id = {k:v for k,v in zip(latters, range(len(latters)))}
id2latters = {v:k for k,v in latters2id.items()}

In [7]:
chunk = pd.read_parquet('./test_data/test_data_'+str(1)+'.pq')

In [8]:
#TODO
# write test data into train directory with dumb labels for further transductive autoencoder pretraining 
for chunk_i in tqdm(range(2)):

    chunk = pd.read_parquet('./test_data/test_data_'+str(chunk_i)+'.pq')

    ids = chunk[["id"]].copy()
    ids["counts"] = 1
    ids = ids.groupby(by="id").sum().reset_index()
    ids = ids.sort_values(by="id")
    labels = [' 2']*len(ids)

    feature_columns = list(chunk.columns.values)
    feature_columns.remove("id")
    feature_columns.remove("rn")

    series = chunk.groupby("id").apply(lambda df:credit2string(df, id2latters, feature_columns ))
    series = series.sort_index()
    
    st = '\n'.join((p+s for p,s in zip(series, labels)))

    fs = bz2.compress(bytearray(st, encoding='ascii'))

    with open(os.path.join("nn_data","clear_test","chunk"+str(chunk_i+12)+".bz2"), "wb") as f:
        f.write(fs)

  0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
y = pd.read_csv('./train_target.csv')

In [11]:
#convert train data
for chunk_i in tqdm(range(12)):

    chunk = pd.read_parquet('./train_data/train_data_'+str(chunk_i)+'.pq')

    ids = chunk[["id"]].copy()
    ids["counts"] = 1
    ids = ids.groupby(by="id").sum().reset_index()
    ids = ids.merge(y, on='id', how='left')
    ids = ids.sort_values(by="id")
    labels = ids.flag.apply(lambda a: ' '+str(a)).tolist()

    feature_columns = list(chunk.columns.values)
    feature_columns.remove("id")
    feature_columns.remove("rn")

    series = chunk.groupby("id").apply(lambda df:credit2string(df, id2latters, feature_columns ))
    series = series.sort_index()
    
    st = '\n'.join((p+s for p,s in zip(series, labels)))

    fs = bz2.compress(bytearray(st, encoding='ascii'))

    with open(os.path.join("nn_data","clear","chunk"+str(chunk_i)+".bz2"), "wb") as f:
        f.write(fs)

  0%|          | 0/12 [00:00<?, ?it/s]

In [12]:
#convert test data
for chunk_i in tqdm(range(2)):

    chunk = pd.read_parquet('./test_data/test_data_'+str(chunk_i)+'.pq')

    feature_columns = list(chunk.columns.values)
    feature_columns.remove("id")
    feature_columns.remove("rn")

    series = chunk.groupby("id").apply(lambda df:credit2string(df, id2latters, feature_columns ))
    series = series.sort_index()
    
    st = '\n'.join(series)
    
    fs = bz2.compress(bytearray(st, encoding='ascii'))

    with open(os.path.join("nn_data","test","chunk"+str(chunk_i)+".bz2"), "wb") as f:
        f.write(fs)

  0%|          | 0/2 [00:00<?, ?it/s]

Make categorical features to ids mapping.

In [3]:
global_mapping = {}

MaxTr = -1
for chunk_i in tqdm(range(12)):

    chunk = pd.read_parquet('./train_data/train_data_'+str(chunk_i)+'.pq')
    df=chunk
    
    feature_columns = list(df.columns.values)
    feature_columns.remove("id")
    feature_columns.remove("rn")
    MaxTr = max(MaxTr, chunk.id.max())
    
    for ft in feature_columns:
        if(ft in global_mapping):
            global_mapping[ft] = global_mapping[ft] | (set(df[ft]))
        else:
            global_mapping[ft] = set(df[ft])

for chunk_i in tqdm(range(2)):

    chunk = pd.read_parquet('./test_data/test_data_'+str(chunk_i)+'.pq')
    df=chunk
    
    feature_columns = list(df.columns.values)
    feature_columns.remove("id")
    feature_columns.remove("rn")
    MaxTr = max(MaxTr, chunk.id.max())
    
    for ft in feature_columns:
        if(ft in global_mapping):
            global_mapping[ft] = global_mapping[ft] | (set(df[ft]))
        else:
            global_mapping[ft] = set(df[ft])

for ft in tqdm(feature_columns):
    mapping = {}
    for k, v in zip(global_mapping[ft], range(len(global_mapping[ft]))):
        mapping[k] = v
    global_mapping[ft] = mapping.copy()
    
feat2id = {k:v for k, v in zip(feature_columns, range(len(feature_columns)))}
id2feat = {v:k for k, v in zip(feature_columns, range(len(feature_columns)))}

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/59 [00:00<?, ?it/s]

In [4]:
with open('feat2id.pickle', 'wb') as handle:
    pickle.dump(feat2id, handle)

with open('id2feat.pickle', 'wb') as handle:
    pickle.dump(id2feat, handle)

with open('global_mapping.pickle', 'wb') as handle:
    pickle.dump(global_mapping, handle)